# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [128]:
import pandas as pd

Скачиваем данные в переменную data

In [129]:
data = pd.read_csv('/datasets/data.csv')

Исследуем общую информацию о датасете, применив методы head, info

In [130]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [132]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Из вызова метода info() мы видим, что в датасете 21525 строк, в двух из столбцов: **total_income**, **days_employed** меньше ненулевых значений, необходимо исследовать эти столбцы

Из вызова метода describe() видим, что в столбцах children, days_employed есть, dob_years присутствуют аномалии: максимальное значение days_employed - 1100 лет

### Шаг 2.1 Заполнение пропусков

In [133]:
non_null_days_employed_ratio = data['days_employed'].isna().sum() / data.shape[0]
print('{:.0%}'.format(non_null_days_employed_ratio))

10%


In [134]:
non_null_total_income_ratio = data['total_income'].isna().sum() / data.shape[0]
print('{:.0%}'.format(non_null_total_income_ratio))

10%


Доля пропущенных значений в обоих столбцах одинакова и составляет 10%.
Рассмотрим подробнее строки с пропущенныхми значениями:

In [135]:
null_values_days_employed = data[data['days_employed'].isna()==True]
null_values_days_employed 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [136]:
null_values_total_income = data[data['total_income'].isna()==True]
null_values_total_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Получили, что в **2174** строках пропущены значения в каждом из столбцов
Проверим, что в тех строках, в которых нет данных по отработанным дням, нет и данных по суммарному доходу:

In [137]:
data[(data['days_employed'].isna() & data['total_income'].isna())==True].shape[0]

2174

Как видим, в тех строках, где отсутствуют значения по отработанным дням, отсутсвуют значения и по суммарному доходу

Сохраним таблицу с пропущенными данными в переменную null_values

In [138]:
null_values = data[(data['days_employed'].isna() & data['total_income'].isna())==True]

In [139]:
null_values

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Проверим, как распределены строки с пропущенными значениями по разным категориям дохода ('income_type'), чтобы оценить, насколько пропуски случайны

In [140]:
nulls = null_values['income_type'].value_counts().reset_index().rename(columns={'index':'income_type','income_type':'count'})

In [141]:
nulls

,income_type,count
0,сотрудник,1105
1,компаньон,508
2,пенсионер,413
3,госслужащий,147
4,предприниматель,1


In [142]:
null_observations = nulls.shape[0]
null_observations

5

В основном датафрейме data распределение следующее:

In [143]:
all_data_income = data['income_type'].value_counts().reset_index().rename(columns={'index':'income_type','income_type':'count'})

In [144]:
observations = data.shape[0]

In [145]:
nulls['ratio'] = nulls['count'].astype('int') / null_observations
nulls.sort_values(by='ratio',ascending=False)

,income_type,count,ratio
0,сотрудник,1105,221.0
1,компаньон,508,101.6
2,пенсионер,413,82.6
3,госслужащий,147,29.4
4,предприниматель,1,0.2


In [146]:
all_data_income['ratio'] = all_data_income['count'].astype('int') / observations
all_data_income.sort_values(by='ratio',ascending=False)

,income_type,count,ratio
0,сотрудник,11119,0.516562
1,компаньон,5085,0.236237
2,пенсионер,3856,0.179141
3,госслужащий,1459,0.067782
4,безработный,2,0.000093
5,предприниматель,2,0.000093
6,студент,1,0.000046
7,в декрете,1,0.000046


Видим, что распредление профессий сходное в срезе с пропущенными данными и в основном датафрейме, скорее всего потеря данных случайна и вызвана человеческим фактором.
В этом случае, для количественных переменных оптимальным будет заполение пропусков медианным значением по категории, это не приведет к сильному искажению данных.

In [147]:
days_employed_income_type_median = data.groupby('income_type')['days_employed'].median()
days_employed_income_type_median

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

In [148]:
total_income_type_median = dict(data.groupby('income_type')['total_income'].median())
total_income_type_median

{'безработный': 131339.7516762103,
 'в декрете': 53829.13072905995,
 'госслужащий': 150447.9352830068,
 'компаньон': 172357.95096577113,
 'пенсионер': 118514.48641164352,
 'предприниматель': 499163.1449470857,
 'сотрудник': 142594.39684740017,
 'студент': 98201.62531401133}

In [149]:
for key, value in days_employed_income_type_median.items():
    data.loc[(data['income_type']==key)&(data['days_employed'].isna()),'days_employed'] = value

In [150]:
for key, value in total_income_type_median.items():
    data.loc[(data['income_type']==key)&(data['total_income'].isna()),'total_income'] = value

In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Как видим, все пропуски заполнены

### Шаг 2.2 Проверка данных на аномалии и исправления.

В ходе работы с датасетом заметили, что в данных столбца days_employed присутствуют отрицательные значения, даже медианное значение по этому столбцу отрицательное. Это явная ошибка внесения или обработки данных.

Изменим отрицательные значения в данном столбце на положительные, заодно заменив тип данных на целочисленный:

In [152]:
data['days_employed'].value_counts()

-1574.202821      1105
-1547.382223       509
 365213.306266     414
-2689.368353       147
-520.848083          2
                  ... 
-339.153770          1
-2849.351119         1
-5619.328204         1
-448.829898          1
-582.538413          1
Name: days_employed, Length: 19353, dtype: int64

In [153]:
data['days_employed'] = abs(data['days_employed']).astype('int')

Проверим, что в столбце не осталось отрицательных значений:

In [154]:
data[data['days_employed'] < 0].shape[0]

0

Максимальное значение столбца days_employed - явный выброс - это 1100 лет работы:

In [155]:
data['days_employed'].max()/365

1100.6986301369864

In [156]:
data['days_employed'].quantile(.9)/365

999.23397260274

Для значений стобца "days_employed" превышающих 50 лет работы, заменим значения на медианные

In [160]:
data.loc[data['days_employed']/365 >=50, 'days_employed'] = median_employed


In [161]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,2230.554332,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,1991.775100,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.000000,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1993.000000,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,2569.000000,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,17615.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Рассмотрим столбец dob_years - показывающий возраст заемщика. Как видим из вызова метода describe() минимельное значение равно 0, необходимо посчитать сколько таких значений и заменить на медианное

In [352]:
(data['dob_years'] == 0).sum()

101

In [353]:
(data['dob_years'] < 18).sum()

101

Заемщику не может быть меньше 18 лет, все знчения возраста меньше 18 нулевые.
Заменим их медианным значением

In [354]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].median()

In [355]:
data['dob_years'].describe()

count    21525.000000
mean        43.490453
std         12.218595
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Как видим, нулевые значения исправлены


Рассмотрим столбец education:

In [356]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [361]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Неявные дубликаты в столбце 'education' удалены

In [362]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [363]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Дубликатов нет

Проверим, какие значения присутствуют в столбце children

In [60]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как видим, в столбце children также присутсвуют аномалии: в 47 строках присутствуют отрицательные значения, в 76 строках указано значение 20 детей. Это очень похоже на ошибку ввода данных, возможно вместо 2 детей указано 20, требует уточнения у поставщика данных.

Обработаем аномальные значения: отрицательное количество детей заменим на положительное, 20 детей заменим на 2

In [61]:
data['children'] = abs(data['children'])

Проверим, что в столбце не осталось отрицательных значений:

In [62]:
data[data['children'] < 0].shape[0]

0

Заменим количество детей с 20 на 2:

In [63]:
data.loc[data['children']==20, 'children'] = 2

Проверим распределение значение в столбце children после обработки аномалий:

In [64]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Аномалии в данном столбце устранены

Проверка столбца total_income на аномалии:
Проверим, какой максимальный доход заемщика в миллионах рублей

In [371]:
data['total_income'].max()/1000000

2.265604028722744

Цифра вполне реальная, выбросов в этом столбце нет

### Шаг 2.3. Изменение типов данных.

В столбце total_income присутсвуют вещественнные значения дохода, удобнее было бы использовать целочисленные значения.

Преобразуем тим данных в столбце total_income:

In [65]:
data['total_income'] = data['total_income'].astype('int')

Проверим тип данных в столбце total_income:

In [66]:
data['total_income'].dtype

dtype('int64')

### Шаг 2.4. Удаление дубликатов.

#### Устранение явных дубликатов

Проверим данные на наличие явных дубликатов:

In [67]:
data.duplicated().sum()

54

Найдено 54 явных дубликата,

Удалим задвоенные строки:

In [68]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим, что явные дубликаты успешно устранены:

In [69]:
data.duplicated().sum()

0

#### Устранение неявных дубликатов

Рассмотрим уникальные значения столбца education

In [70]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Как видим одни и те же значения написаны в разном регистре.

Приведем все значения в столбце education к нижнему регистру:

In [71]:
data['education'] = data['education'].str.lower()

Проверим уникальные значения столбца education после преобразования:

In [72]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Рассмотрим уникальные значения столбца family_status:

In [73]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В данном столбце неявные дубликаты отсутсвуют

Рассмотрим уникальные значения столбца income_type:

In [74]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В данном столбце неявные дубликаты отсутсвуют

Рассмотрим уникальные значения столбца gender:

In [75]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В данном столбце неявные дубликаты отсутсвуют  
  
  

Рассмотрим уникальные значения столбца purpose:

In [76]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце purpose присутсвует множество неявных дубликатов, например:
- 'покупка жилья', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка жилой недвижимости', 'покупка своего жилья'
- 'свадьба', 'сыграть свадьбу', 'на проведение свадьбы'
- 'приобретение автомобиля', 'на покупку подержанного автомобиля' и 

Обработку дубликатов в этом столбце **проведем НИЖЕ с помощью функции make_purpose_category**

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [77]:
education = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
family = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

In [78]:
education.head()

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [79]:
family.head()

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удалим столбцы education и family_status

In [80]:
data = data.drop(columns=['education', 'family_status'], axis=1)

### Шаг 2.6. Категоризация дохода.

Создадим функцию для определения категорий на основе дохода

In [81]:
def make_income_category(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'

Создадим новый столбец в датафрейме - total_income_category

In [83]:
data['total_income_category'] = data['total_income'].apply(make_income_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.0,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.0,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.0,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Проверим корректность:


In [85]:
data.total_income_category.value_counts()

C    16033
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

После категоризации проверим данные на явныые дубликаты:

In [364]:
data.duplicated().sum()

72

Удалим явные дубликаты:

In [367]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим, что явные дубликаты удалены:

In [368]:
data.duplicated().sum()

0

### Шаг 2.7. Категоризация целей кредита.

Создадим фукнцию **make_purpose_category**, которая объединит различные значения столбца purpose в слудующие категории: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования'

In [88]:
def make_purpose_category(row):
    if 'автомоб' in row:
        return 'операции с автомобилем'
    elif ('недвижим' in row) or ('квартир' in row) or ('жиль' in row):
        return 'операции с недвижимостью'
    elif 'свадьб' in row:
        return 'проведение свадьбы'
    else: 
        return 'получение образования'

Применим функцию make_purpose_category для формирования нового столбца pupose_category в датафрейме

In [89]:
data['purpose_category'] = data['purpose'].apply(make_purpose_category)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.0,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.0,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.0,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.0,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.0,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.0,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.0,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.0,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Проверим корректность категоризации:


In [90]:
data['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [47]:
data_child = data.loc[:, ['children', 'debt']]

In [48]:
data_child_indebt = data_child.groupby('children')['debt'].sum().reset_index()
data_child_total = data_child.groupby('children').count().reset_index()

Посчитаем количество невозвратов кредитов в каждой группе по детям

In [49]:
data_child_indebt

,children,debt
0,0,1063
1,1,445
2,2,202
3,3,27
4,4,4
5,5,0


Посчитаем количество заявок в каждой группе по де

In [50]:
data_child_total

,children,debt
0,0,14107
1,1,4856
2,2,2128
3,3,330
4,4,41
5,5,9


Объединим данные в одну таблицу и посчитаем ratio - отношение невозвратов к общему количеству заявок в каждой категории

In [51]:
data_children_merged = data_child_total.merge(data_child_indebt, on='children')
data_children_merged = data_children_merged.rename(columns = {'debt_x': 'total', 'debt_y': 'indebt'})
data_children_merged['ratio'] = (data_children_merged['indebt'] / data_children_merged['total']).round(decimals=4)
data_children_merged = data_children_merged.sort_values(by = 'ratio', ascending=False).reset_index(drop=True)
data_children_merged

,children,total,indebt,ratio
0,4,41,4,0.0976
1,2,2128,202,0.0949
2,1,4856,445,0.0916
3,3,330,27,0.0818
4,0,14107,1063,0.0754
5,5,9,0,0.0000


Как видно из таблицы, самый большой риск невозврата у семей с 4 и 2 детьми, а самые аккуратные заемщики - бездетные, все 5-детные семьи кредиты вернули, но их было всего 9 на всю выборку, что не позволяет делать уверенные выводы

**Или более простым способом:**

In [52]:
data_child.pivot_table(index='children',values='debt',aggfunc='mean').sort_values(by='debt', ascending=False).round(decimals=3)

,debt
children,
4,0.098
2,0.095
1,0.092
3,0.082
0,0.075
5,0.000


##### Вывод 1:  
Как видно из таблицы, самый большой риск невозврата у семей с 4 и 2 детьми, а самые аккуратные заемщики - бездетные, все 5-детные семьи кредиты вернули, но их было всего 9 на всю выборку, что не позволяет делать уверенные выводы

##### Вопрос 2:  
**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [53]:
data_family = data.loc[:, ['family_status_id', 'debt']]
data_family_pivot = data_family.pivot_table(index='family_status_id',values='debt',aggfunc='mean').sort_values(by='debt',ascending=False).round(decimals=3)
data_family_pivot.merge(family, on='family_status_id')

,family_status_id,debt,family_status
0,4,0.098,Не женат / не замужем
1,1,0.093,гражданский брак
2,0,0.075,женат / замужем
3,3,0.071,в разводе
4,2,0.066,вдовец / вдова


##### Вывод 2:  
Как видно из таблицы, зависимость между семейным положением и невозвратом кредитав в срок есть: самый большой процент невозвратов у неженатых и живущих гражданским браком, а самый низкий у вдовцов

##### Вопрос 3:  
**Есть ли зависимость между целями кредита и возвратом кредита в срок?**

Создадим таблицу целей и возвратов кредита

In [54]:
data_purpose = data.loc[:, ['purpose_category','debt']]

In [55]:
data_purpose.pivot_table(index='purpose_category', values='debt',aggfunc='mean').sort_values(by='debt',ascending=False).round(decimals=3)

,debt
purpose_category,
операции с автомобилем,0.094
получение образования,0.092
проведение свадьбы,0.080
операции с недвижимостью,0.072


##### Вывод 3:  
Как видно из таблицы, зависимость между целью кредита и процентом невозврата в срок  есть: операции с автомобилем и получение образования - самые проблемные

## Общий вывод:

Кредитный отдел банка анализирует информацию о качестве заемщиков, используя множество параметров, таких как  семейное положение, количество детей, уровень образования, уровень дохода и пр.
  
В данной работе рассматривалось одного параметра на возврат кредита в срок. Выяснено, что и количество детей, и уровень дохода, и семейное положение, и цели кредита влияют на аккуратность возврата кредита.
  
В реальности, требуется провести более сложный анализ взаимовлияния этих факторов на качество заемщика.